In [3]:
suppressMessages(library(data.table))
suppressMessages(library(dplyr))

filename = "covid19_snps/COVID19_HGI_A2_ALL_leave_23andme_20210107.b37.txt"
covid_gwas = fread(filename, data.table = F)

message(paste("starting snps:", nrow(covid_gwas)))

starting snps: 7542185



In [5]:
message("removing duplicate snps ==========")
dup_snps = covid_gwas$SNP[which(duplicated(covid_gwas$SNP) == T)]
covid_gwas_filtered = covid_gwas[!covid_gwas$SNP %in% dup_snps,]
message(paste("snps left:", nrow(covid_gwas_filtered)))

message("removing ambiguous snps ==========")
ambig_snps = covid_gwas_filtered[(covid_gwas_filtered$REF == "A" & covid_gwas_filtered$ALT == "T") | 
                                 (covid_gwas_filtered$REF == "T" & covid_gwas_filtered$ALT == "A") |
                                 (covid_gwas_filtered$REF == "G" & covid_gwas_filtered$ALT == "C") |
                                 (covid_gwas_filtered$REF == "C" & covid_gwas_filtered$ALT == "G") ,]
covid_gwas_filtered = covid_gwas_filtered[!covid_gwas_filtered$SNP %in% ambig_snps$SNP,]
message(paste("snps left:", nrow(covid_gwas_filtered)))

message("labeling lead snps ========")
leads = c("3:45899651:A:G", "19:4719443:G:A", "21:34615210:T:C", "12:113381956:C:T", "17:47940666:C:T", "6:31121232:T:C", "7:54647894:A:C")
colnames(covid_gwas_filtered)[1] = "CHR"
covid_gwas_filtered %>% arrange(all_inv_var_meta_p) %>% filter(SNP %in% leads)
covid_gwas_filtered$is_lead = ifelse(covid_gwas_filtered$SNP %in% leads, TRUE, FALSE)

fwrite(covid_gwas_filtered, "covid19_snps/COVID19_HGI_A2_filtered.txt", row.names = F, sep = "\t")
write.table(covid_gwas_filtered$SNP, "COVID19_HGI_A2_filtered.sigsnps", row.names = F, sep = "\t", col.names = F, quote =  F)


removing duplicate snps ==========

snps left: 7542185

removing ambiguous snps ==========

snps left: 6442586

labeling lead snps ========



CHR,POS,REF,ALT,SNP,all_meta_N,all_inv_var_meta_beta,all_inv_var_meta_sebeta,all_inv_var_meta_p,all_inv_var_het_p,all_meta_sample_N,all_meta_AF,rsid
<int>,<int>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<chr>
3,45899651,A,G,3:45899651:A:G,16,0.63527,0.043464,2.219e-48,0.0548100,711838,0.07506,rs34326463
19,4719443,G,A,19:4719443:G:A,18,0.24627,0.026120,4.174e-21,0.3561000,714592,0.32790,rs2109069
21,34615210,T,C,21:34615210:T:C,18,-0.20152,0.026152,1.301e-14,0.6625000,714592,0.66230,rs13050728
12,113381956,C,T,12:113381956:C:T,18,0.18936,0.026007,3.308e-13,0.8415000,714592,0.67330,rs2269899
6,31121232,T,C,6:31121232:T:C,18,0.27497,0.039770,4.713e-12,0.0001754,714592,0.09981,rs111837807
17,47940666,C,T,17:47940666:C:T,16,0.39990,0.069472,8.597e-09,0.5526000,710955,0.03468,rs77534576
7,54647894,A,C,7:54647894:A:C,14,0.19114,0.033706,1.422e-08,0.0001394,472724,0.14830,rs622568
